## Создание таблицы факторов по кукам для кред скоринга
### Расчет на бою

In [3]:
import datetime

In [4]:
try:
    sc.stop()
except:
    pass

In [28]:
class calc_cred_score():

    init_query = '''
    set hive.vectorized.execution.enabled=true;
    set mapreduce.map.memory.mb=4096;
    set mapreduce.map.child.java.opts=-Xmx4g;
    set mapreduce.task.io.sort.mb=1024;
    set mapreduce.reduce.child.java.opts=-Xmx4g;
    set mapreduce.reduce.memory.mb=7000;
    set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
    set mapreduce.input.fileinputformat.split.minsize=536870912;
    set mapreduce.input.fileinputformat.split.maxsize=1073741824;
    set hive.optimize.ppd=true;
    set hive.merge.smallfiles.avgsize=536870912;
    set hive.merge.mapredfiles=true;
    set hive.merge.mapfiles=true;
    set hive.hadoop.supports.splittable.combineinputformat=true;
    set hive.exec.reducers.bytes.per.reducer=536870912;
    set hive.exec.parallel=true;
    set hive.exec.max.created.files=10000000;
    set hive.exec.compress.output=true;
    set hive.exec.dynamic.partition.mode=nonstrict;
    set hive.exec.max.dynamic.partitions=1000000;
    set hive.exec.max.dynamic.partitions.pernode=100000;
    set io.seqfile.compression.type=BLOCK;
    set mapred.output.compression.codec=org.apache.hadoop.io.compress.BZip2Codec;
    set mapreduce.map.failures.maxpercent=5;

    set hive.tez.auto.reducer.parallelism=true;
    set hive.tez.min.partition.factor=0.25;
    set hive.tez.max.partition.factor=2.0;
    set tez.runtime.pipelined.sorter.lazy-allocate.memory=true;
    set hive.exec.dynamic.partition.mode=nonstrict;
    '''

    create_accum_query = '''
    CREATE TABLE IF NOT EXISTS `user_kposminin.id_feat_accum`(
          `id` string, 
          `load_src` string, 
          `first_id_ymd` string, 
          `last_id_ymd` string, 
          `ymd_cnt` int, 
          `urlfr_cnt` bigint, -- `cnt` bigint, 
          `visits_cnt` bigint, 
          `hits` bigint, 
          `emailru_sum` bigint, 
          `mobile_sum` double, 
          `vk_sum` double, 
          `social_sum` double, 
          `work_hours_hits_sum` double, 
          `avg_hour_sum_sq` bigint, 
          `avg_hour_sum` bigint, 
          `max_score1` double, 
          `max_score2` double, 
          `max_score3` double, 
          `max_score4` double, 
          `max_score5` double, 
          `max_score6` double, 
          `max_score7` double, 
          `max_score8` double, 
          `min_score1` double, 
          `min_score2` double, 
          `min_score3` double, 
          `min_score4` double, 
          `min_score5` double, 
          `min_score6` double, 
          `min_score7` double, 
          `min_score8` double, 
          `sum_score1` double, 
          `sum_score2` double, 
          `sum_score3` double, 
          `sum_score4` double, 
          `sum_score5` double, 
          `sum_score6` double, 
          `sum_score7` double, 
          `sum_score8` double, 
          `cnt_score1` int, 
          `cnt_score2` int, 
          `cnt_score3` int, 
          `cnt_score4` int, 
          `cnt_score5` int, 
          `cnt_score6` int, 
          `cnt_score7` int, 
          `cnt_score8` int, 
          `good_urlfr_sum_score1` double, 
          `good_urlfr_sum_score2` double, 
          `good_urlfr_sum_score3` double,
          `good_urlfr_sum_score4` double,
          `good_urlfr_sum_score5` double,
          `good_urlfr_sum_score6` double,
          `good_urlfr_sum_score7` double,
          `good_urlfr_sum_score8` double)
    partitioned by (
          `first_calc_ymd` string, 
          `last_calc_ymd` string      
          )
    ;

    '''
 
    feat_1d_query_pattern = '''

    -- #ymd. Id_feats 1 day calc
    with mymd as 
     (select
       target,
       max(ymd) as max_ymd
      from
       user_kposminin.urlfr_scores
      where 
        ymd < date_add('#ymd',-3)
      group by target
     )

    insert overwrite table user_kposminin.id_feat_accum partition (first_calc_ymd, last_calc_ymd)
    select 
     id,
     load_src, 
     ymd as first_id_ymd, 
     ymd as last_id_ymd,
     1 as ymd_cnt,
     count(distinct urlfr) as urlfr_cnt,
     count(urlfr) as visits_cnt,
     sum(cnt) as hits,
     sum(if(urlfr like 'e.mail.ru%',1,0)) as emailru_sum,
     sum(if(urlfr like 'm.%',1,0)) as mobile_sum,
     sum(if(urlfr rlike '^(m\\.)?vk.com', 1, 0)) as vk_sum,
     sum(if(urlfr rlike '^(m\\.)?vk.com' or urlfr rlike '^(m\\.)?(ok|odnoklassniki)\\.ru' or urlfr rlike '^(m\\.)?my.mail.ru',1,0)) as social_sum,
     sum(if(avg_hour >= 9 and avg_hour <= 20,cnt,0)) as work_hours_hits_sum,
     sum( avg_hour * avg_hour) as avg_hour_sum_sq,
     sum(avg_hour) as avg_hour_sum,
     max(score1) as max_score1,
     max(score2) as max_score2,
     max(score3) as max_score3,
     max(score4) as max_score4,
     max(score5) as max_score5,
     max(score6) as max_score6,
     max(score7) as max_score7,
     max(score8) as max_score8,
     min(score1) as min_score1,
     min(score2) as min_score2,
     min(score3) as min_score3,
     min(score4) as min_score4,
     min(score5) as min_score5,
     min(score6) as min_score6,
     min(score7) as min_score7,
     min(score8) as min_score8,
     sum(score1) as sum_score1,
     sum(score2) as sum_score2,
     sum(score3) as sum_score3,
     sum(score4) as sum_score4, 
     sum(score5) as sum_score5,
     sum(score6) as sum_score6, 
     sum(score7) as sum_score7,
     sum(score8) as sum_score8, 
     count(score1) as cnt_score1,
     count(score2) as cnt_score2,
     count(score3) as cnt_score3,
     count(score4) as cnt_score4, 
     count(score5) as cnt_score5,
     count(score6) as cnt_score6, 
     count(score7) as cnt_score7,
     count(score8) as cnt_score8, 
     count( if(score1 > -0.2, urlfr,Null)) as good_urlfr_sum_score1,
     count( if(score2 > -9, urlfr,Null)) as good_urlfr_sum_score2,
     count( if(score3 > -9, urlfr,Null)) as good_urlfr_sum_score3,
     count( if(score4 > -2, urlfr,Null)) as good_urlfr_sum_score4,
     count( if(score5 > -9, urlfr,Null)) as good_urlfr_sum_score5,
     count( if(score6 > -0.2, urlfr,Null)) as good_urlfr_sum_score6,
     count( if(score7 > -9, urlfr,Null)) as good_urlfr_sum_score7,
     count( if(score8 > -0.2, urlfr,Null)) as good_urlfr_sum_score8,
     '#ymd' as first_calc_ymd, 
     '#ymd' as last_calc_ymd
 
    from
     (select
        v.id,
        v.load_src as load_src,
        v.ymd,
        v.url_fragment as urlfr,
        unix_timestamp(v.ymd, 'yyyy-MM-dd')/60/60 + v.average_visit_hour  as time_h,
        1 as time_std,
        v.visit_count as cnt,
        v.average_visit_hour as avg_hour,
        t1.score as score1,
        t2.score as score2,
        t3.score as score3,
        t4.score as score4,
        t5.score as score5,
        log((t2.cnt_positive + 0.1)/(t3.cnt_positive - t2.cnt_positive + 0.1)) as score6,
        t7.score as score7,
        log((t5.cnt_positive + 0.1)/(t7.cnt_positive - t5.cnt_positive + 0.1)) as score8
      from
        prod_odd.visit_feature v
        left semi join (
          select 
            uid_str as id,
            property_value as phone_num,
            load_src
          from
            prod_dds.md_uid_property 
          where
            property_cd = 'PHONE'
          ) m on m.id = v.id and m.load_src = v.load_src
        left join (
            select urlfr,score
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'ccall_tinkoff_approve_from_fullapp'
        ) t1 on t1.urlfr = v.url_fragment
        left join (
            select urlfr,score,positive as cnt_positive
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'tinkoff_platinum_approved_application03@tinkoff_action' 
        ) t2 on t2.urlfr = v.url_fragment
        left join (
            select urlfr,score,positive as cnt_positive
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'tinkoff_platinum_complete_application03@tinkoff_action'
        ) t3 on t3.urlfr = v.url_fragment
        left join (
            select urlfr,score
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'tinkoff_LON_CCR_default'
        ) t4 on t4.urlfr = v.url_fragment
        left join (
            select urlfr,score,positive as cnt_positive
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'tinkoff_platinum_approved_application03_1m'
        ) t5 on t5.urlfr = v.url_fragment
        left join (
            select urlfr,score,positive as cnt_positive
              from mymd td
             inner join user_kposminin.urlfr_scores t on t.target = td.target and t.ymd = td.max_ymd
             where td.target = 'tinkoff_platinum_complete_application03_1m'
        ) t7 on t7.urlfr = v.url_fragment
      where 
        v.ymd = '#ymd' 
     ) a 
    group by
      id,load_src,ymd
    ;
    '''

    accumulator_merge_pattern = '''

    insert overwrite table user_kposminin.id_feat_accum partition (first_calc_ymd,last_calc_ymd)
    select
      nvl(a.id,b.id) as id,
      nvl(a.load_src,b.load_src) as load_src,
      least(a.first_id_ymd,b.first_id_ymd) as first_id_ymd,
      greatest(a.last_id_ymd,b.last_id_ymd) as last_id_ymd,
      nvl(a.ymd_cnt,0) + nvl(b.ymd_cnt,0) as ymd_cnt,
      nvl(a.urlfr_cnt,0) + nvl(b.urlfr_cnt,0) as urlfr_cnt,
      nvl(a.visits_cnt,0) + nvl(b.visits_cnt,0) as visits_cnt,
      nvl(a.hits,0) + nvl(b.hits,0) as hits,
      nvl(a.emailru_sum,0) + nvl(b.emailru_sum,0) as emailru_sum,
      nvl(a.mobile_sum,0) + nvl(b.mobile_sum,0) as mobile_sum,
      nvl(a.vk_sum,0) + nvl(b.vk_sum,0) as vk_sum,
      nvl(a.social_sum,0) + nvl(b.social_sum,0) as social_sum,
      nvl(a.work_hours_hits_sum,0) + nvl(b.work_hours_hits_sum,0) as work_hours_hits_sum,
      nvl(a.avg_hour_sum_sq,0) + nvl(b.avg_hour_sum_sq,0) as avg_hour_sum_sq,
      nvl(a.avg_hour_sum,0) + nvl(b.avg_hour_sum,0) as avg_hour_sum,
      greatest(a.max_score1,b.max_score1) as max_score1,
      greatest(a.max_score2,b.max_score2) as max_score2,
      greatest(a.max_score3,b.max_score3) as max_score3,
      greatest(a.max_score4,b.max_score4) as max_score4,
      greatest(a.max_score5,b.max_score5) as max_score5,
      greatest(a.max_score6,b.max_score6) as max_score6,
      greatest(a.max_score7,b.max_score7) as max_score7,
      greatest(a.max_score8,b.max_score8) as max_score8,
      least(a.min_score1, b.min_score1) as min_score1,
      least(a.min_score2, b.min_score2) as min_score2,
      least(a.min_score3, b.min_score3) as min_score3,
      least(a.min_score4, b.min_score4) as min_score4,
      least(a.min_score5, b.min_score5) as min_score5,
      least(a.min_score6, b.min_score6) as min_score6,
      least(a.min_score7, b.min_score7) as min_score7,
      least(a.min_score8, b.min_score8) as min_score8,
      nvl(a.sum_score1,0) + nvl(b.sum_score1,0) as sum_score1,
      nvl(a.sum_score2,0) + nvl(b.sum_score2,0) as sum_score2,
      nvl(a.sum_score3,0) + nvl(b.sum_score3,0) as sum_score3,
      nvl(a.sum_score4,0) + nvl(b.sum_score4,0) as sum_score4,
      nvl(a.sum_score5,0) + nvl(b.sum_score5,0) as sum_score5,
      nvl(a.sum_score6,0) + nvl(b.sum_score6,0) as sum_score6,
      nvl(a.sum_score7,0) + nvl(b.sum_score7,0) as sum_score7,
      nvl(a.sum_score8,0) + nvl(b.sum_score8,0) as sum_score8,
      nvl(a.cnt_score1,0) + nvl(b.cnt_score1,0) as cnt_score1,
      nvl(a.cnt_score2,0) + nvl(b.cnt_score2,0) as cnt_score2,
      nvl(a.cnt_score3,0) + nvl(b.cnt_score3,0) as cnt_score3,
      nvl(a.cnt_score4,0) + nvl(b.cnt_score4,0) as cnt_score4,
      nvl(a.cnt_score5,0) + nvl(b.cnt_score5,0) as cnt_score5,
      nvl(a.cnt_score6,0) + nvl(b.cnt_score6,0) as cnt_score6,
      nvl(a.cnt_score7,0) + nvl(b.cnt_score7,0) as cnt_score7,
      nvl(a.cnt_score8,0) + nvl(b.cnt_score8,0) as cnt_score8,
      nvl(a.good_urlfr_sum_score1,0) + nvl(b.good_urlfr_sum_score1,0) as good_urlfr_sum_score1,
      nvl(a.good_urlfr_sum_score2,0) + nvl(b.good_urlfr_sum_score2,0) as good_urlfr_sum_score2,
      nvl(a.good_urlfr_sum_score3,0) + nvl(b.good_urlfr_sum_score3,0) as good_urlfr_sum_score3,
      nvl(a.good_urlfr_sum_score4,0) + nvl(b.good_urlfr_sum_score4,0) as good_urlfr_sum_score4,
      nvl(a.good_urlfr_sum_score5,0) + nvl(b.good_urlfr_sum_score5,0) as good_urlfr_sum_score5,
      nvl(a.good_urlfr_sum_score6,0) + nvl(b.good_urlfr_sum_score6,0) as good_urlfr_sum_score6,
      nvl(a.good_urlfr_sum_score7,0) + nvl(b.good_urlfr_sum_score7,0) as good_urlfr_sum_score7,
      nvl(a.good_urlfr_sum_score8,0) + nvl(b.good_urlfr_sum_score8,0) as good_urlfr_sum_score8,
      '#new_first_calc_ymd' as first_calc_ymd,
      '#new_last_calc_ymd' as last_calc_ymd
    from 
      (select * from user_kposminin.id_feat_accum a where a.first_calc_ymd = '#a_first_calc_ymd' and a.last_calc_ymd = '#a_last_calc_ymd') a
      full join (select * from user_kposminin.id_feat_accum b where b.first_calc_ymd = '#b_first_calc_ymd' and b.last_calc_ymd = '#b_last_calc_ymd') b 
        on a.id = b.id and a.load_src = b.load_src
    ;


    '''

    clear_partition_query = '''
    alter table user_kposminin.id_feat_accum drop partition (first_calc_ymd = '#first_calc_ymd', last_calc_ymd = '#last_calc_ymd');
    '''

    create_feat_table_query = '''

    create table user_kposminin.id_feat_ccall as
    select 
         a.phone_mobile, 
         count(distinct a.id) as id_cnt,
         count(distinct acc.id) as acc_id_cnt,
         a.call_ymd as call_ymd,
         max(a.approve) as approve,
         sum(acc.urlfr_cnt) as urlfr_cnt,
         sum(acc.visits_cnt) as visits_cnt, 
         sum(acc.hits) as hits,
         max(acc.max_score1) as max_score1,
         sum(acc.sum_score1) / sum(acc.cnt_score1) as avg_score1,
         min(acc.min_score1) as min_score1,
         max(acc.max_score2) as max_score2,
         sum(acc.sum_score2) / sum(acc.cnt_score2) as avg_score2,
         min(acc.min_score2) as min_score2,
         max(acc.max_score3) as max_score3,
         sum(acc.sum_score3) / sum(acc.cnt_score3) as avg_score3,
         min(acc.min_score3) as min_score3,
         max(acc.max_score4) as max_score4,
         sum(acc.sum_score4) / sum(acc.cnt_score4) as avg_score4,
         min(acc.min_score4) as min_score4,
         max(acc.max_score5) as max_score5,
         sum(acc.sum_score5) / sum(acc.cnt_score4) as avg_score5,
         min(acc.min_score5) as min_score5,
         max(acc.max_score6) as max_score6,
         sum(acc.sum_score6) / sum(acc.cnt_score4) as avg_score6,
         min(acc.min_score6) as min_score6,
         sum(acc.emailru_sum) / sum(acc.visits_cnt) as emailru_share,
         sum(acc.mobile_sum) / sum(acc.visits_cnt) as mobile_share,
         sum(acc.vk_sum) / sum(acc.visits_cnt) as vk_share,
         sum(acc.social_sum) / sum(acc.visits_cnt) as social_share,
         sum(acc.work_hours_hits_sum) / sum(acc.hits) as work_hours_hits_share,
         sqrt(sum(acc.avg_hour_sum_sq)/(sum(acc.visits_cnt)-1) - power(sum(acc.avg_hour_sum)/(sum(acc.visits_cnt) - 1), 2)) as hour_std,
         sum(acc.good_urlfr_sum_score1) / sum(acc.cnt_score1) as good_urlfr_share_score1, 
         sum(acc.good_urlfr_sum_score2) / sum(acc.cnt_score2) as good_urlfr_share_score2,
         sum(acc.good_urlfr_sum_score3) / sum(acc.cnt_score3) as good_urlfr_share_score3, 
         sum(acc.good_urlfr_sum_score4) / sum(acc.cnt_score4) as good_urlfr_share_score4, 
         sum(acc.good_urlfr_sum_score5) / sum(acc.cnt_score5) as good_urlfr_share_score5,
         sum(acc.good_urlfr_sum_score6) / sum(acc.cnt_score6) as good_urlfr_share_score6,
         max(trim(pc.provider)) as mob_provider,
         max(r.ind) as ind,
         max(r.pop_country_share) as pop_country_share, 
         max(r.pop_city_share) as pop_city_share, 
         max(r.population / r.area_sq_km) as density,
         max(r.area_sq_km) as area_sq_km,
         max(trim(r.federal_district)) as federal_district, 
         max(r.avg_salary_2015_rub) as avg_salary_2015_rub, 
         max(r.utc_time_zone_val) as utc_time_zone_val


      from
         user_kposminin.ccall_aza_id a
         inner join user_kposminin.id_feat_accum acc on acc.id = a.id and acc.last_calc_ymd = date_add(a.call_ymd, -1)
         left join dds_dic.phone_codes pc on trim(pc.phone_code) = substr(a.phone_mobile,2,9)
         left join dds_dic.region_stat r on r.ind = pc.region_id
     group by
             a.phone_mobile,
             a.call_ymd
    ;

    '''
    
    def __init__(self, hc = None, n_threads = 10, init_query = True):
        import datetime
        if(init_query):
            self.query = self.init_query + self.create_accum_query
        else:
            self.query = ''            
        self.log = ''
        
    def calc_days(self, days, merge = True, clean = True):
        ''' Calc 1d feat and accum it for days (datetime.datetime list)'''
        days = sorted(days)
        acc_first_calc_ymd, acc_last_calc_ymd = [days[0]] * 2
        for day in days:
            new_first_calc_ymd = min(day,acc_first_calc_ymd)
            new_last_calc_ymd  = max(day,acc_last_calc_ymd)
            
            self.query += self.feat_1d_query_pattern.replace('#ymd',day.strftime('%Y-%m-%d'))
            if merge & (days.index(day) > 0):
                self.query += (self.accumulator_merge_pattern 
                       .replace('#a_first_calc_ymd', acc_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#a_last_calc_ymd', acc_last_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#b_first_calc_ymd', day.strftime('%Y-%m-%d'))
                       .replace('#b_last_calc_ymd', day.strftime('%Y-%m-%d'))
                       .replace('#new_first_calc_ymd', new_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#new_last_calc_ymd',  new_last_calc_ymd.strftime('%Y-%m-%d'))
                               )
                if clean & (days.index(day) > 1):
                    self.query += (self.clear_partition_query 
                       .replace('#first_calc_ymd', acc_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#last_calc_ymd', acc_last_calc_ymd.strftime('%Y-%m-%d'))
                                  )
                if ((clean == 'full_clean') & (days.index(day) > 1)): 
                    self.query += (self.clear_partition_query 
                       .replace('#first_calc_ymd', day.strftime('%Y-%m-%d'))
                       .replace('#last_calc_ymd', day.strftime('%Y-%m-%d'))
                                  )
            acc_first_calc_ymd = new_first_calc_ymd
            acc_last_calc_ymd = new_last_calc_ymd
    
    def calc_day_range(self, first_day = None, last_day = None, n_days = None, merge = True, clean = True):        
        assert (first_day is None) + (last_day is None) + (n_days is None) == 1, '''calc_day_range Error:
              exactly two of three params must be filled: first_day, last_day, n_days'''.replace('\n',' ')
        if first_day is None:
            day_range = [last_day + datetime.timedelta(days = i) for i in range(-n_days + 1,1)]
        elif last_day is None:
            day_range = [first_day + datetime.timedelta(days = i) for i in range(n_days)]
        else:
            first_day,last_day = min(first_day,last_day), max(first_day,last_day)
            day_range = [first_day + datetime.timedelta(days = i) for i in range((last_day - first_day).days + 1)]
        self.calc_days(day_range, merge, clean)
        
    def get_query(self):
        return self.query

    def merge_days(self, day_tuples, clean = False):
        ''' day_tuples is a list of (first_day,last_day) to merge'''
        day_tuples = sorted(day_tuples)
        acc_first_calc_ymd, acc_last_calc_ymd = day_tuples[0]
        first = True
        for (first_day,last_day) in day_tuples[1:]:
            new_first_calc_ymd = min(first_day,acc_first_calc_ymd)
            new_last_calc_ymd  = max(last_day,acc_last_calc_ymd)
            self.query += (self.accumulator_merge_pattern 
                       .replace('#a_first_calc_ymd', acc_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#a_last_calc_ymd',   acc_last_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#b_first_calc_ymd', first_day.strftime('%Y-%m-%d'))
                       .replace('#b_last_calc_ymd',   last_day.strftime('%Y-%m-%d'))
                       .replace('#new_first_calc_ymd', new_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#new_last_calc_ymd',   new_last_calc_ymd.strftime('%Y-%m-%d'))
                               )
            if (not clean is False) & ((not first) or (clean == 'full_clean')):
                self.query += (self.clear_partition_query 
                       .replace('#first_calc_ymd', acc_first_calc_ymd.strftime('%Y-%m-%d'))
                       .replace('#last_calc_ymd',   acc_last_calc_ymd.strftime('%Y-%m-%d'))
                                  )
            if (clean == 'full_clean'): 
                self.query += (self.clear_partition_query 
                       .replace('#first_calc_ymd', first_day.strftime('%Y-%m-%d'))
                       .replace('#last_calc_ymd',   last_day.strftime('%Y-%m-%d'))
                                  )
            acc_first_calc_ymd = new_first_calc_ymd
            acc_last_calc_ymd = new_last_calc_ymd
            first = False
            
    def calc_full_feat(self):
        self.query += self.create_feat_table_query
        
    def execute_query(self, hc):
        '''Execute query using hc HiveContext'''
        for q in self.query.split(';'):
            if re.search('[^ \t\n]',q):
                self.log += 'Executing {}\n'.format(q)
                hc.sql(q)
    
    def get_log(self):
        return log


In [23]:
#! nohup /opt/anaconda/bin/python cred_scor_feat_table_calc_4.py 2016-12-01 2016-12-15 > cred_scor_feat_table_calc_27.log 2>&1 &
#os.popen('beeline -u "jdbc:hive2://ds-hadoop-cs01p:10000/" -n kposminin -e "' + ' '.join(c.get_query().split(';')[30].split('\n')[5:])+ '"').read()
from pyspark import SparkConf, SparkContext, HiveContext
import re
conf = (SparkConf()
        .set("spark.executor.instances", 2)
        .set("spark.driver.maxResultSize", "4g")
        .set('spark.driver.memory','4g')
        .set("spark.executor.memory", '2g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )

try:
    sc.stop()
except:
    pass
sc = SparkContext()
hc = HiveContext(sc)

DataFrame[_c0: int]

In [29]:
query0 = '''
WITH t AS 
 (
   SELECT
   v.url_fragment AS urlfr
   ,count(distinct if(ta.ymd between v.ymd and date_add(v.ymd,3),ta.id,Null)) as cnt_positive
   ,count(distinct v.id) as cnt_total
  FROM
   prod_odd.visit_feature v
   left join (
     SELECT DISTINCT a.id,a.ymd 
     FROM prod_features_liveinternet.user_action a 
     WHERE a.ymd between date_add('2017-03-01', -30) and date_add('2017-03-01',3) 
       and a.action_type = 'tinkoff_platinum_complete_application'
   ) ta on v.id = ta.id
  WHERE
   v.ymd between date_add('2017-03-01', -30) and '2017-03-01'
   and v.load_src = 'LI.02'
  GROUP BY 
   v.url_fragment
   ) 
INSERT OVERWRITE TABLE 
  user_kposminin.urlfr_tgt_cnt PARTITION (ymd='2017-03-01', target='tinkoff_platinum_complete_application03_1m') 
SELECT 
 urlfr AS urlfr
 ,nvl(cnt_positive, 0) as cnt_positive
 ,cnt_total
 ,log((cnt_positive + 0.1)/(cnt_total - cnt_positive + 0.1)) as score
FROM t 
;

insert overwrite table user_kposminin.urlfr_scores partition (ymd, target)
select
  urlfr,
  cnt_positive as positive,
  cnt_total as total,
  score as score,
  ymd,
  target
from user_kposminin.urlfr_tgt_cnt
where
  target = 'tinkoff_platinum_complete_application03_1m'
  and ymd = '2017-03-01'
  and (cnt_total > 30000 or cnt_positive > 20)
;

WITH t AS 
 (
   SELECT
   v.url_fragment AS urlfr
   ,count(distinct if(ta.ymd between v.ymd and date_add(v.ymd,3),ta.id,Null)) as cnt_positive
   ,count(distinct v.id) as cnt_total
  FROM
   prod_odd.visit_feature v
   left join (
     SELECT DISTINCT a.id,a.ymd 
     FROM prod_features_liveinternet.user_action a 
     WHERE a.ymd between date_add('2017-03-01', -30) and date_add('2017-03-01',3) 
       and a.action_type = 'tinkoff_platinum_approved_application'
   ) ta on v.id = ta.id
  WHERE
   v.ymd between date_add('2017-03-01', -30) and '2017-03-01'
   and v.load_src = 'LI.02'
  GROUP BY 
   v.url_fragment
   ) 
INSERT OVERWRITE TABLE 
  user_kposminin.urlfr_tgt_cnt PARTITION (ymd='2017-03-01', target='tinkoff_platinum_approved_application03_1m') 
SELECT 
 urlfr AS urlfr
 ,nvl(cnt_positive, 0) as cnt_positive
 ,cnt_total
 ,log((cnt_positive + 0.1)/(cnt_total - cnt_positive + 0.1)) as score
FROM t 
;

insert overwrite table user_kposminin.urlfr_scores partition (ymd, target)
select
  urlfr,
  cnt_positive as positive,
  cnt_total as total,
  score as score,
  ymd,
  target
from user_kposminin.urlfr_tgt_cnt
where
  target = 'tinkoff_platinum_approved_application03_1m'
  and ymd = '2017-03-01'
  and (cnt_total > 30000 or cnt_positive > 10)
;

'''

In [55]:
import datetime
c = calc_cred_score()
#c.calc_day_range(first_day = datetime.datetime(2016,12,1) ,last_day = datetime.datetime(2016,12,31),merge = True, clean = True)
c.merge_days([(datetime.datetime(2017,5,1),datetime.datetime(2017,5,15)),(datetime.datetime(2017,5,16),datetime.datetime(2017,5,31))], clean = True)
#c.merge_days([(datetime.datetime(2017,4,1),datetime.datetime(2017,4,30)),(datetime.datetime(2017,5,1),datetime.datetime(2017,5,15)),(datetime.datetime(2017,5,16),datetime.datetime(2017,5,31))], clean = True)
#c.merge_days([(datetime.datetime(2017,3,16),datetime.datetime(2017,3,31)),(datetime.datetime(2017,4,1),datetime.datetime(2017,4,30)),
#              (datetime.datetime(2017,4,1),datetime.datetime(2017,4,15))], clean = False)
#c.merge_days([(datetime.datetime(2017,3,1),datetime.datetime(2017,3,15)),(datetime.datetime(2017,3,16),datetime.datetime(2017,4,30))], clean = True)

print(c.get_query())

#print('-'*100)
#c.execute_query(hc)


    set hive.vectorized.execution.enabled=true;
    set mapreduce.map.memory.mb=4096;
    set mapreduce.map.child.java.opts=-Xmx4g;
    set mapreduce.task.io.sort.mb=1024;
    set mapreduce.reduce.child.java.opts=-Xmx4g;
    set mapreduce.reduce.memory.mb=7000;
    set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
    set mapreduce.input.fileinputformat.split.minsize=536870912;
    set mapreduce.input.fileinputformat.split.maxsize=1073741824;
    set hive.optimize.ppd=true;
    set hive.merge.smallfiles.avgsize=536870912;
    set hive.merge.mapredfiles=true;
    set hive.merge.mapfiles=true;
    set hive.hadoop.supports.splittable.combineinputformat=true;
    set hive.exec.reducers.bytes.per.reducer=536870912;
    set hive.exec.parallel=true;
    set hive.exec.max.created.files=10000000;
    set hive.exec.compress.output=true;
    set hive.exec.dynamic.partition.mode=nonstrict;
    set hive.exec.max.dynamic.partitions=1000000;
    set hive.exec.max.dynamic.partitions.pernode=1000

#### Заполняем промежутки по 15 дней. В итоге весь год посчитан. История за 2 месяца.

In [ ]:

day_tuples = sorted([(datetime.datetime(2016,i,1),datetime.datetime(2016,i,15)) for i in range(1,13)] + 
             [(datetime.datetime(2016,i,16),datetime.datetime(2016,i+1,1) - datetime.timedelta(days=1)) for i in range(1,12)] + 
             [(datetime.datetime(2016,12,16),datetime.datetime(2016,12,31))])
import datetime
for i in range(len(day_tuples) - 4):
    c = calc_cred_score(hc, init_query = False)
    c.merge_days(day_tuples[i:i+4], clean = 'clean')
    day = day_tuples[i+3][1] + datetime.timedelta(days=1)
    inner_day_tuple = [(min([e[0] for e in day_tuples[i:i+4]]),max([e[1] for e in day_tuples[i:i+4]]))]
    while day < day_tuples[i+4][1]:
        inner_day_tuple.append((day,day))
        day += datetime.timedelta(days=1)
    c.merge_days(inner_day_tuple, clean = False)
    #print(c.get_query())
    print('-'*100)


In [101]:
query = '''create table user_kposminin.ccall_aza_id as 
select distinct phone_mobile, approve, call_ymd, id from user_kposminin.ccall_visits
;'''

c = calc_cred_score(hc, init_query = False)
c.calc_full_feat()
print(c.get_query())




    create table user_kposminin.id_feat_ccall as
    select 
         a.phone_mobile, 
         count(distinct a.id) as id_cnt,
         count(distinct acc.id) as acc_id_cnt,
         a.call_ymd as call_ymd,
         max(a.approve) as approve,
         sum(acc.urlfr_cnt) as urlfr_cnt,
         sum(acc.visits_cnt) as visits_cnt, 
         sum(acc.hits) as hits,
         max(acc.max_score1) as max_score1,
         sum(acc.sum_score1) / sum(acc.cnt_score1) as avg_score1,
         min(acc.min_score1) as min_score1,
         max(acc.max_score2) as max_score2,
         sum(acc.sum_score2) / sum(acc.cnt_score2) as avg_score2,
         min(acc.min_score2) as min_score2,
         max(acc.max_score3) as max_score3,
         sum(acc.sum_score3) / sum(acc.cnt_score3) as avg_score3,
         min(acc.min_score3) as min_score3,
         max(acc.max_score4) as max_score4,
         sum(acc.sum_score4) / sum(acc.cnt_score4) as avg_score4,
         min(acc.min_score4) as min_score4,
         max(acc.ma

## Таблица построена. Проверяем работоспособность - строим классификатор

In [4]:
##### Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''

sc.stop()
conf = (SparkConf()
        .set("spark.executor.instances", 2)
        .set("spark.driver.maxResultSize", "4g")
        .set('spark.driver.memory','4g')
        .set("spark.executor.memory", '2g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

In [5]:

def metrics(y_true,y_score,lift = None, return_str = False):
    import sklearn
    import collections
    
    if True:
        
        res = collections.OrderedDict()
        samp_size = len(y_true)
        res['Sample size'] = samp_size
        res['Posit share'] = sum(y_true) * 1./ samp_size
        res['Sample size'] = len(y_true)
        res['AUC ROC'] = sklearn.metrics.roc_auc_score(y_true = y_true, y_score = y_score)
        res['AUC PR'] = sklearn.metrics.average_precision_score( y_true,  y_score)
        res['Log loss'] = sklearn.metrics.log_loss(y_true = y_true, y_pred = y_score)
        if lift:
            predictions_and_labels = sorted(zip(y_score,y_true), key = lambda e:-e[0])
            for l in lift:
                res['Lift ' + str(l)] = sum([e[1] for e in predictions_and_labels[:int(l * samp_size)]]) * 1. / int(l * samp_size) / res['Posit share']                
        if return_str:
            res = '\n'.join(['{:<12}: {:.5f}'.format(k,v) for (k,v) in res.items()]) + '.'
        return res

In [6]:
def cv(clf, X, y, folds = 5, metrics = 'roc_auc,pr_auc'):
    '''calc cross-validation metrics for clf classfier on X,y data.
       clf must support predict_proba method.
       Returns dictionary with metrics values on each fold.
    '''
    assert X.shape[0] == len(y), 'X and y lengths doesnt match'
    idx = range(X.shape[0])
    np.random.shuffle(idx)
    fold_idx = []
    for i in range(folds):
        fold_idx.append(idx[i*len(idx)/folds:(i+1)*len(idx)/folds])
    res = {k:[] for k in metrics.split(',')}
    
    for i in range(folds):
        train_idx = reduce(lambda x,y: x+y,(fold_idx[:i] + fold_idx[(i+1):]))
        valid_idx = fold_idx[i]
        
        clf.fit(X[train_idx],y[train_idx])
        valid_pred = clf.predict_proba(X[valid_idx])[:,1]
        
        if('roc_auc' in metrics):
            res['roc_auc'].append(
                sklearn.metrics.roc_auc_score(
                  y_true = y[valid_idx],
                  y_score = valid_pred
                )
            )
        if('pr_auc' in metrics):
            res['pr_auc'].append(
                sklearn.metrics.average_precision_score(
                  y_true = y[valid_idx],
                  y_score = valid_pred
                )
            )
    return res

In [7]:
import cPickle
from sklearn.feature_extraction import DictVectorizer
import os
import sklearn, sklearn.cross_validation
from pylightgbm.models import GBMClassifier

federal_districts = [u'ЦФО',u'СЗФО',u'ЮФО',u'СКФО',u'ПФО',u'УФО',u'СФО',u'ДВФО'] # todo

def encode(v, classes, default_value = -1):
    '''Encode text value v which values are from classes list. Returns v index and -1 if it wasn't found in the list.'''
    try:
        return classes.index(v)
    except ValueError:
        return default_value


exec_path = "/opt/share/LightGBM-master/lightgbm"
os.environ["LIGHTGBM_EXEC"] = exec_path

In [8]:
df_all = (hc.sql('select * from user_kposminin.id_feat_ccall')        
        .toPandas()
         )

#df_all.loc[:,'federal_district'] = df_all.federal_district.map(lambda v:encode(v, federal_districts))
df_train = df_all[df_all['call_ymd'] <  '2016-12-01']
df_test  = df_all[df_all['call_ymd'] >= '2016-12-01']
df_train.shape,df_test.shape

((333259, 47), (28946, 47))

In [9]:
from sklearn import preprocessing
categ_vars = [ 'mob_provider',u'federal_district']
le = {v:preprocessing.LabelEncoder() for v in categ_vars}
for v in categ_vars:
    df_train.loc[:,v] = le[v].fit_transform(df_train[v])
    df_test.loc[:,v]  = le[v].transform(df_test[v])

/opt/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
#df_all.iloc[0]

In [10]:
def target_encoding(cat_vars, label, df_train, df_test, folds = 5, smooth_N = 100, inplace = False):
    '''
       Make target encoding of categorical_variable from cat_vars list using label.
       df_train, df_test are Pandas.DataFrame instances.
       folds is number of folds to split.
       smooth_N is smoothing number.
       returns Pandas DataFrames with categorical_variable + '_encoded' columns for train and test.
    '''
    assert type(df_train) == type(df_test) == type(pd.DataFrame()), 'df_train and df_test must be Pandas DataFrame.'
    assert all(v in df_train.columns for v in cat_vars + [label]) & \
           all(v in df_test.columns for v in cat_vars + [label]),   \
           'all categorical variables and label must be df_train and df_test columns'
    
    idx = df_train.index.tolist()
    np.random.shuffle(idx)
    fold_idx = []    
    for i in range(folds):
        fold_idx.append(idx[i*len(idx)/folds:(i+1)*len(idx)/folds])
    
    df_train_encoded = pd.DataFrame(index = idx, columns = [v + '_encoded' for v in cat_vars])
    df_test_encoded = pd.DataFrame(index = df_test.index, columns = [v + '_encoded' for v in cat_vars])
    
    for i in range(folds):
        train_idx = reduce(lambda x,y: x+y,(fold_idx[:i] + fold_idx[(i+1):]))
        valid_idx = fold_idx[i]
        
        global_mean = df_train.ix[train_idx,label].mean()
        for v in cat_vars:
            print(v)
            v_stat = df_train.ix[train_idx].groupby(v)[label].agg([sum,len])
            df_train_encoded.ix[valid_idx,v + '_encoded'] = df_train.ix[valid_idx,v].map( lambda c:
             (v_stat.loc[c,'sum'] + smooth_N * global_mean) / (v_stat.loc[c,'len'] + smooth_N) if c in v_stat.index else global_mean)
    
    for v in cat_vars:
        v_stat = df_train.groupby(v)[label].agg([sum,len])
        df_test_encoded.loc[:,v + '_encoded'] = df_test.loc[:,v].map( lambda c:
             (v_stat.loc[c,'sum'] + smooth_N * global_mean) / (v_stat.loc[c,'len'] + smooth_N) if c in v_stat.index else global_mean)
        
    if inplace == True:
        df_train = df_train.join(df_train_encoded)
        df_test = df_test.join(df_test_encoded)
    else:
        return df_train_encoded, df_test_encoded


In [11]:
df_train_enc,df_test_enc =  target_encoding(cat_vars = [ 'mob_provider',u'federal_district'], label = 'approve', df_train = df_train, df_test = df_test, folds = 3, smooth_N = 100)
#df_train.columns
df_train = df_train.join(df_train_enc)
df_test  = df_test.join(df_test_enc)

mob_provider
federal_district
mob_provider
federal_district
mob_provider
federal_district


In [207]:
feat_list = [
    df_train.columns[5:],
    [c for c in df_train.columns[5:] if not 'score5' in c], # good ??
    [c for c in df_train.columns[5:] if not 'score4' in c], 
    [c for c in df_train.columns[5:] if not 'score1' in c], 
    [c for c in df_train.columns[5:] if not any('score{}'.format(i) in c for i in list('236'))],
    df_train.columns[5:-11], # bad
    df_train.columns[5:-17], # bad
    df_train.columns[5:]  + df_train.columns[1:2],
    [c for c in (df_train.columns[5:]  + df_train.columns[1:2]) if not 'score5' in c], # good ??
    [c for c in (df_train.columns[5:-2]  + df_train.columns[1:2]) if not 'score5' in c],
    df_train.columns[5:-2] + df_train.columns[1:2],   
    [c for c in (df_train.columns[5:]  + df_train.columns[1:2]) if not 'share' in c]
]

label = 'approve'


In [166]:
print(datetime.datetime.now())
result = []
for i in range(len(feat_list)):
    clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
        verbose = False
    )
    result.append((i,feat_list[i], cv(clf, df_train[feat_list[i]].values, df_train[label].values, folds = 5, metrics = 'roc_auc,pr_auc')))
print(datetime.datetime.now())

2017-05-31 13:52:58.993424
2017-05-31 14:27:04.858684


In [173]:
df_res = pd.DataFrame(result,columns = ['id','feats','metrics'])
df_res['avg_auc_roc'] = df_res['metrics'].map(lambda d: np.mean(d['roc_auc']))
df_res['std_auc_roc'] = df_res['metrics'].map(lambda d: np.std(d['roc_auc']))
df_res['min_auc_roc'] = df_res['metrics'].map(lambda d: min(d['roc_auc']))
df_res['avg_auc_pr'] = df_res['metrics'].map(lambda d: np.mean(d['pr_auc']))
df_res

,id,feats,metrics,avg_auc_roc,std_auc_roc,min_auc_roc,avg_auc_pr
0,0,"Index([ u'urlfr_cnt', ...","{u'pr_auc': [0.511794013588, 0.513386253207, 0...",0.668986,0.001475,0.667104,0.516982
1,1,"[urlfr_cnt, visits_cnt, hits, max_score1, avg_...","{u'pr_auc': [0.515478496553, 0.51499617237, 0....",0.669223,0.001317,0.668141,0.516767
2,2,"[urlfr_cnt, visits_cnt, hits, max_score1, avg_...","{u'pr_auc': [0.516846995118, 0.512782025885, 0...",0.668198,0.001627,0.665775,0.515902
3,3,"[urlfr_cnt, visits_cnt, hits, max_score2, avg_...","{u'pr_auc': [0.511834229821, 0.506280271948, 0...",0.659296,0.002111,0.657187,0.508054
4,4,"[urlfr_cnt, visits_cnt, hits, max_score1, avg_...","{u'pr_auc': [0.513497564319, 0.521698216584, 0...",0.666274,0.001960,0.664360,0.514638
5,5,"Index([u'urlfr_cnt', u'visits_cnt', u'hits', u...","{u'pr_auc': [0.506971324264, 0.503659634598, 0...",0.658319,0.001968,0.655521,0.504773
6,6,"Index([u'urlfr_cnt', u'visits_cnt', u'hits', u...","{u'pr_auc': [0.503215024669, 0.505294663653, 0...",0.656823,0.001969,0.654138,0.503058
7,7,"Index([ u'urlfr_cnt', ...","{u'pr_auc': [0.514410032847, 0.51649660086, 0....",0.669756,0.002703,0.667237,0.517546


### Исключение score5 улучшает модель (?), добавление id_cnt тоже улучшает.

In [205]:
print('Feature importances')
sorted(zip(sorted(clf.feature_importance().items()),yet_another_feat_list[-1]),key = lambda v:-v[0][1])

[((2, 754), 'avg_score1'),
 ((38, 739), 'urlfr_cnt'),
 ((27, 735), 'min_score1'),
 ((5, 615), 'avg_score4'),
 ((18, 547), 'hour_std'),
 ((40, 535), 'visits_cnt'),
 ((13, 517), 'good_urlfr_share_score3'),
 ((17, 460), 'hits'),
 ((23, 459), 'max_score3'),
 ((4, 438), 'avg_score3'),
 ((11, 401), 'good_urlfr_share_score1'),
 ((3, 394), 'avg_score2'),
 ((33, 386), 'mob_provider'),
 ((6, 362), 'avg_score5'),
 ((37, 361), 'social_share'),
 ((35, 354), 'pop_city_share'),
 ((7, 353), 'avg_score6'),
 ((14, 348), 'good_urlfr_share_score4'),
 ((41, 346), 'vk_share'),
 ((1, 345), 'avg_salary_2015_rub'),
 ((30, 342), 'min_score4'),
 ((25, 341), 'max_score5'),
 ((34, 338), 'mobile_share'),
 ((16, 337), 'good_urlfr_share_score6'),
 ((42, 315), 'work_hours_hits_share'),
 ((31, 304), 'min_score5'),
 ((21, 301), 'max_score1'),
 ((22, 299), 'max_score2'),
 ((29, 293), 'min_score3'),
 ((32, 283), 'min_score6'),
 ((0, 278), 'area_sq_km'),
 ((15, 278), 'good_urlfr_share_score5'),
 ((9, 274), 'emailru_share')

## Вывод: берем все факторы

## Далее тюним параметры

In [15]:
feats = df_train.columns[1:2] + df_train.columns[5:]
label = 'approve'

#### ставим высокий learning rate и определяем для него число деревьев

In [ ]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=100,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.1,
        metric = 'auc',
    )
res = []
for n_trees in [30,50,100,150,250]:
    clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=100,
       # is_unbalance = True,
        num_iterations = n_trees,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.1,
        metric = 'auc',
        verbose = False
    )
    res.append((n_trees, cv(clf, df_train[feats].values, df_train[label].values, folds = 5, metrics = 'roc_auc,pr_auc')))  

print('\n'.join('{} {}'.format(e[0],np.mean(e[1]['roc_auc'])) for e in res))

#df_res = pd.DataFrame(result,columns = ['id','feats','metrics'])
#df_res['avg_auc_roc'] = df_res['metrics'].map(lambda d: np.mean(d['roc_auc']))
#df_res['std_auc_roc'] = df_res['metrics'].map(lambda d: np.std(d['roc_auc']))
#df_res['min_auc_roc'] = df_res['metrics'].map(lambda d: min(d['roc_auc']))
#df_res['avg_auc_pr'] = df_res['metrics'].map(lambda d: np.mean(d['pr_auc']))
#df_res

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading data in 4.245451 seconds
[LightGBM] [Info] Number of postive: 97346, number of negative: 169262
[LightGBM] [Info] Number of data: 266608, number of features: 45
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] 0.301915 seconds elapsed, finished iteration 1
[LightGBM] [Info] 0.559169 seconds elapsed, finished iteration 2
[LightGBM] [Info] 0.803832 seconds elapsed, finished iteration 3
[LightGBM] [Info] 1.015187 seconds elapsed, finished iteration 4
[LightGBM] [Info] 1.237167 seconds elapsed, finished iteration 5
[LightGBM] [Info] 1.452098 seconds elapsed, finished iteration 6
[LightGBM] [Info] 1.674369 seconds elapsed, finished iteration 7
[LightGBM] [Info] 1.902531 seconds elapsed, finished iteration 8
[LightGBM] [Info] 2.152716 seconds elapsed, finished iteration 9
[LightGBM] [Info] 2.512865 seconds elapsed, finished iteration 10
[LightGBM] [Info] 2.

In [209]:
param_grid = {
    'min_data_in_leaf': [10,50,100,200,500],
    'num_leaves': [15,31,63,127,255],
    'bagging_fraction': [0.5,0.8, 1],
    'bagging_freq': [5,10,30],
    'feature_fraction': [0.5, 0.8, 1]
}

clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf = 100,
       # is_unbalance = True,
        num_iterations = 70,
        bagging_fraction = 1,
        bagging_freq = 10,
        num_leaves = 15,
        learning_rate = 0.1,
        feature_fraction = 0.8,
        metric = 'auc',
        verbose=False
    )

param_set = (
    {'learning_rate':0.1, 'num_iterations':50, 'num_leaves':127},
    {'learning_rate':0.1, 'num_iterations':70, 'num_leaves':15},
    {'learning_rate':0.08, 'num_iterations':100, 'num_leaves':15},
    {'learning_rate':0.08, 'num_iterations':100, 'num_leaves':31},
    {'learning_rate':0.06, 'num_iterations':150, 'num_leaves':31},
    {'learning_rate':0.06, 'num_iterations':150, 'num_leaves':15},
    {'learning_rate':0.05, 'num_iterations':200, 'num_leaves':31},
    {'learning_rate':0.04, 'num_iterations':200, 'num_leaves':63},
    {'learning_rate':0.04, 'num_iterations':300, 'num_leaves':63},
    {'learning_rate':0.03, 'num_iterations':400, 'num_leaves':31},
    {'learning_rate':0.05, 'num_iterations':300, 'num_leaves':31},
)


#n_iter = 60
res= []
for param in param_set:
    clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf = 100,
        # is_unbalance = True,
        num_iterations = param['num_iterations'],
        bagging_fraction = 1,
        bagging_freq = 10,
        num_leaves = param['num_iterations'],
        learning_rate = param['learning_rate'],
        feature_fraction = 0.8,
        metric = 'auc',
        verbose=False
    )
    
    res.append((param,cv(clf, df_train[feats].values, df_train[label].values, folds = 5, metrics = 'roc_auc,pr_auc')))


Index([                  u'id_cnt',                u'urlfr_cnt',
                     u'visits_cnt',                     u'hits',
                     u'max_score1',               u'avg_score1',
                     u'min_score1',               u'max_score2',
                     u'avg_score2',               u'min_score2',
                     u'max_score3',               u'avg_score3',
                     u'min_score3',               u'max_score4',
                     u'avg_score4',               u'min_score4',
                     u'max_score5',               u'avg_score5',
                     u'min_score5',               u'max_score6',
                     u'avg_score6',               u'min_score6',
                  u'emailru_share',             u'mobile_share',
                       u'vk_share',             u'social_share',
          u'work_hours_hits_share',                 u'hour_std',
        u'good_urlfr_share_score1',  u'good_urlfr_share_score2',
        u'good_urlfr_shar

In [121]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
    )
#clf.fit(df_train[feats], df_train[label])
#df_test['pred'] = clf.predict_proba(df_test[feats])[:,1]

clf.fit(df_train[feats], df_train[label], test_data = [(df_test[feats], df_test[label])])

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading data in 5.608001 seconds
[LightGBM] [Info] Number of postive: 121590, number of negative: 211669
[LightGBM] [Info] Number of data: 333259, number of features: 42
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] Iteration:1, valid_1 auc : 0.658906
[LightGBM] [Info] 0.269187 seconds elapsed, finished iteration 1
[LightGBM] [Info] Iteration:2, valid_1 auc : 0.662137
[LightGBM] [Info] 0.546721 seconds elapsed, finished iteration 2
[LightGBM] [Info] Iteration:3, valid_1 auc : 0.665258
[LightGBM] [Info] 0.841890 seconds elapsed, finished iteration 3
[LightGBM] [Info] Iteration:4, valid_1 auc : 0.666226
[LightGBM] [Info] 1.256363 seconds elapsed, finished iteration 4
[LightGBM] [Info] Iteration:5, valid_1 auc : 0.668245
[LightGBM] [Info] 1.621473 seconds elapsed, finished iteration 5
[LightGBM] [Info] Iteration:6, valid_1 auc : 0.668524
[LightGBM] [Info] 1.9

# Это успех

In [141]:
df_test['pred'] = clf.predict_proba(df_test[feats])[:,1]
#print(metrics(y_true = df_test[label], y_score = df_test.pred, lift = None, return_str = True))

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


In [130]:
feats = [f for f in feats if not f == 'mob_provider']

In [142]:
df_another_test = (hc.sql('select * from user_kposminin.id_feat_20170429')        
        .toPandas()
         )
for v in categ_vars:
    df_another_test.loc[:,v] = le[v].fit_transform(df_another_test[v])
df_another_test['pred'] = clf.predict_proba(df_another_test[feats])[:,1]
print(metrics(y_true = df_another_test[label], y_score = df_another_test.pred, lift = None, return_str = True))

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction
Sample size : 1928.00000
Posit share : 0.48081
AUC ROC     : 0.64123
AUC PR      : 0.59594
Log loss    : 0.69476.


### Немного маловато. С чем это связано: устаревание модели, неконсистентность данных, выброс за счет малой выборки либо заявки немного разные (обучение + первый тест из выборки Азамата, которая неизвестно как сформирована. another_test сформирован из всех заявок по звонкам)?
### Гипотезу малой выборки проверим построением доверительного интервала для AUC ROC бутстреппингом:

In [143]:
sample = np.random.choice(df_another_test.index,3)
roc_auc_bootstrap = []
for _ in range(1000):
    sample_idx = np.random.choice(df_test.index, df_another_test.shape[0], replace = True)
    roc_auc_bootstrap.append(sklearn.metrics.roc_auc_score(y_true = df_test.ix[sample_idx,label], y_score = df_test.ix[sample_idx,'pred']))
print('ROC AUC percentiles {}'.format(np.percentile(roc_auc_bootstrap,q=[5,10,30,50,70,90,95])))    

ROC AUC percentiles [ 0.66129092  0.66584395  0.67504882  0.68097747  0.68805436  0.69782539
  0.70172197]


### Наблюдаемый AUC ROC не влезает даже в 95% доверительный интервал. Итак, низкий AUC ROC не связан с малой выборкой. 
#### Возможно, причина в разнородности заявок в обучении и в последнем тесте .